##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Crie um Estimador com um modelo do Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/keras_model_to_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

> Aviso: os Estimadores não são recomendados para novos códigos.  Os Estimadores executam código `v1.Session`, que é mais difícil de escrever corretamente e pode se comportar de forma inesperada, ainda mais quando usado em conjunto com código do TF 2. Os Estimadores são abarcados pelas [garantias de compatibilidade](https://tensorflow.org/guide/versions), mas não recebem mais correções, exceto para vulnerabilidades de segurança. Confira mais detalhes no [guia de migração](https://tensorflow.org/guide/migrate).

## Visão geral

Os Estimadores do TensorFlow são compatíveis com o TensorFlow e podem ser criados a partir de modelos `tf.keras` novos e existentes. Este tutorial contém um exemplo completo e simples desse processo.

Observação: se você tem um modelo do Keras, pode usá-lo diretamente com as [estratégias `tf.distribute`](https://tensorflow.org/guide/migrate/guide/distributed_training) sem convertê-lo para um Estimador. Portanto, o `model_to_estimator` não é mais recomendado.

## Configuração

In [ ]:
import tensorflow as tf

import numpy as np
import tensorflow_datasets as tfds

### Crie um modelo simples do Keras.

No Keras, você agrupa *camadas* para criar *modelos*. Um modelo é (geralmente) um grafo de camadas. O tipo de modelo mais comum é uma pilha de camadas: o modelo `tf.keras.Sequential`.

Para criar uma rede simples e totalmente conectada, ou seja, um perceptron multicamadas:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])

Compile o modelo e obtenha um resumo.

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam')
model.summary()

### Crie uma função de entrada

Usando a [API Datasets](../../guide/data.md), escale para datasets maiores ou treinamento de vários dispositivos.

Os Estimadores precisam controlar quando e como o pipeline de entrada é criado. Para permitir isso, eles exigem uma "função de entrada" ou `input_fn`. O `Estimator` chamará essa função sem argumentos. O `input_fn` precisa retornar um `tf.data.Dataset`.

In [ ]:
def input_fn():
  split = tfds.Split.TRAIN
  dataset = tfds.load('iris', split=split, as_supervised=True)
  dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels))
  dataset = dataset.batch(32).repeat()
  return dataset

Teste seu `input_fn`

In [ ]:
for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)

### Crie um Estimador com o modelo tf.keras.

Um `tf.keras.Model` pode ser treinado com a API `tf.estimator` ao converter o modelo em um objeto `tf.estimator.Estimator` com `tf.keras.estimator.model_to_estimator`.

In [ ]:
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model, model_dir=model_dir)

Treine e avalie o estimador.

In [ ]:
keras_estimator.train(input_fn=input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))